# Analiza zmiany wartości mid price na podstawie Limit Order Book

Będziemy LOBy analizować 


In [4]:
from imbalance import prep_data
from parser import parse
from utils import confusion_matrix
from warnings import filterwarnings
filterwarnings('ignore')


file_locs = ['data/OrderBookSnapshots.9061.csv', 'data/OrderBookSnapshots.9062.csv', 'data/OrderBookSnapshots.9063.csv',
             'data/OrderBookSnapshots.9064.csv', 'data/OrderBookSnapshots.9065.csv']

## Imbalance

No to z czym sie będziemy porównywać to imbalance z pracy (link)

In [5]:
from sklearn import linear_model


for file_name in file_locs:
    X_train, Y_train, X_test, Y_test = prep_data(parse(file_name))
    classifier = linear_model.SGDClassifier(loss="log", alpha=0.1, max_iter=3000, tol=0, shuffle=False)
    classifier.fit(X_train, Y_train)
    Y_check = classifier.predict(X_test)
    print(classifier.score(X_test,Y_test))
    print(confusion_matrix(Y_check, Y_test))
    print()

0.5435016111707841
[[226 160]
 [265 280]]

0.5314834578441836
[[195 164]
 [275 303]]

0.5532150776053215
[[263 242]
 [161 236]]

0.553038105046344
[[149 101]
 [333 388]]

0.5866141732283464
[[199 136]
 [179 248]]



## Klasyfikator oparty na sieci neuronowej

In [6]:
import torch
from torch import nn
import numpy as np

No nie wiem czy coś tu pisać

### Preprocessing

Zamiast brać słupki z LOB dzielimy cały zakres na kubełki, których wielkość to ułamek wartości mid price. Następnie bierzemy tylko kilka z każdej strony mid price i normujemy tak, aby wartości sumowały się do jedynki.


In [7]:

def fits(index, arr):
    return index >= 0 and index < len(arr)


def get_XY(data, n_buckets=5, bucket_size=0.05 , omit_no_change=True):
    
    keys = list(data.keys())
    keys.sort()

    growths = []
    X = []

    for i, curr in enumerate(keys[:-1]):
        currKey = curr
        nextKey = keys[i+1]

        if not omit_no_change or data[nextKey][2] != data[currKey][2]:
            rows = np.zeros(2*n_buckets)
            mid_price = data[currKey][2]
            #print(mid_price)
            for bid_price, bid_size in reversed(data[currKey][0]):
                bucket = int(( n_buckets*bucket_size -  (mid_price-bid_price)/mid_price)/bucket_size)
                #print(bucket)
                if fits(bucket, rows):
                    rows[bucket] += bid_size

            for ask_price, ask_size in data[currKey][1]:
                bucket = int(((ask_price - mid_price)/mid_price )/bucket_size)
                #print(bucket + n_buckets)
                if fits(bucket + n_buckets, rows):
                    rows[bucket + n_buckets] += ask_size

            ###print("!!!", currKey, nextKey)

            # poprawne dane - min ask > max bid
            if data[currKey][0][-1][0] <  data[currKey][1][0][0] and data[nextKey][0][-1][0] <  data[nextKey][1][0][0]:
                growths.append(data[currKey][2] < data[nextKey][2])
                rows /= rows.sum()
                X.append(rows)
                
    return np.array(X, dtype=np.float32), np.array(growths, dtype=np.int)


### Model

Do trenowania używamy klasycznej sieci neuronowej z warstwami fc i dodatkowo dropoutem

In [8]:
bucket_size = 0.04
input_size = 2*3
hidden_size = 1000

loss = nn.CrossEntropyLoss()

In [9]:
model = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.Tanh(),
        nn.Dropout(p=0.6, inplace=False),
        nn.Linear(hidden_size, hidden_size//4),
        nn.ReLU(),
        nn.Linear(hidden_size//4, 2)
    )

## Wyniki

In [10]:
from utils import accuracy

from preprocess import get_XY

for data in file_locs:
    X, Y = get_XY(parse(data), n_buckets=input_size//2, bucket_size=bucket_size)
    
    set_number = data[data.find('.')+1:data.rfind('.')]
    
    model.load_state_dict(torch.load("models/model"+set_number))
    model.eval()
    X = torch.from_numpy(X)
    Y = torch.from_numpy(Y)
    pred = model(X)
    loss_val = loss(pred, Y)
    print("Loss: {}".format(loss_val.item()))
    print("Acc: {}".format(accuracy(pred.data.numpy(), Y.data.numpy())))
    print("Confusion matrix: ")
    print(confusion_matrix(np.argmax(pred.data.numpy(), axis=1), Y.data.numpy()))
    print()
    
## python3 main.py --data ../data/OrderBookSnapshots.9062.csv --debug --epochs 250 --buckets 3 --bucket_size 0.04 --print_every 100 --hidden 1000

Loss: 0.6941971182823181
Acc: 0.5
Confusion matrix: 
[[  76   76]
 [1780 1780]]

Loss: 0.6952551007270813
Acc: 0.4825910931174089
Confusion matrix: 
[[1281 1376]
 [ 541  507]]

Loss: 0.693829357624054
Acc: 0.5049254151421334
Confusion matrix: 
[[1386 1386]
 [ 373  408]]

Loss: 0.6929921507835388
Acc: 0.512707468879668
Confusion matrix: 
[[1128 1126]
 [ 753  849]]

Loss: 0.6936193108558655
Acc: 0.5059021922428331
Confusion matrix: 
[[935 928]
 [537 565]]



Dziękujemy za uwagę